#### To import LSTM predictions to Ensemble Model

In [ ]:
#Reads and concatenates validation sets. Files that were produced from LSTM training may need to be renamed.
import os
import pandas as pd
path = ".\Dataframes"

df = pd.DataFrame() 
for i in range(10):
    df_i = pd.read_csv(f"0_KKS.csv_{i}.csv", header = None)
    df = df.append(df_i)
df = df.reset_index(drop = True)

colnames = ["pred_outcome","outcome","accuracy","text","docket","newpred"]
df.columns = colnames
df = df.drop(columns=["newpred"])
df.to_csv(f"{path}\\LSTM predictions.csv",index = False)

In [ ]:
#Splits DF into petitioner and respondent, checks reports accuracy for each
def split_pet_resp(df):
    #Splits DF into one for petitioner and one for respondent
    respondent_rows = []
    petitioner_rows = []
    for i, row in df.iterrows():
        if i%2 != 0: #Get only respondent dockets
            respondent_rows.append(i)
        else:
            petitioner_rows.append(i)
    pet_df = df.iloc[petitioner_rows]
    rep_df = df.iloc[respondent_rows]
    
    return pet_df,rep_df

pet_df,rep_df = split_pet_resp(df)
print(check_accuracy(df,"outcome","pred_outcome"))
print(check_accuracy(rep_df,"outcome","pred_outcome"))
print(check_accuracy(pet_df,"outcome","pred_outcome"))

### ADD SOFTMAX PREDS TO KKS MODEL ###
kks_df = pd.read_csv(f"{path}\\feature_table.csv")
kks_df["neuralegal_resp"] = ""
kks_df["neuralegal_pet"] = ""

for i, row in kks_df.iterrows():
    for j, rowj in rep_df.iterrows():
        if row["docket"] == rowj["docket"]:            
            pred = rowj["accuracy"]
            kks_df.at[i,"neuralegal_resp"] = pred
    for k, rowk in pet_df.iterrows():
        if row["docket"] == rowk["docket"]:            
            pred = rowk["accuracy"]
            kks_df.at[i,"neuralegal_pet"] = pred

kks_df.to_csv(f"{path}\\feature_table.csv",index=False)

#### Check AUC on predictions ###

In [2]:
import numpy as np
from sklearn import metrics
import pandas as pd

path = ".\Dataframes"

lstm_df = pd.read_csv(f"{path}\\LSTM predictions.csv")
ensemble_df = pd.read_csv(f"{path}\\ensemble predictions.csv")
KKS_df = pd.read_csv(f"{path}\\results_kks.csv")

def check_accuracy(df,correctrow,predictedrow):
    correct = 0
    for i, row in df.iterrows():
        if row[correctrow] == row[predictedrow]:
            correct +=1
    accuracy = correct/len(df)
    
    return accuracy

def get_AUC(df,outcome_col,pred_col):
    y = []
    scores = []
    for i, row in df.iterrows():
        actual = row[outcome_col]
        score = row[pred_col]
        y.append(actual)
        scores.append(score)
    
    accuracy = check_accuracy(df,outcome_col,pred_col)
    print("Accuracy:", accuracy)
    print("AUC Score:", metrics.roc_auc_score(y, scores))
    print("Classification Report:\n", metrics.classification_report(y,scores))

    return

get_AUC(lstm_df,"outcome","pred_outcome")
get_AUC(ensemble_df,"Actual","Predicted")
get_AUC(KKS_df,"Actual","Predicted")

Accuracy: 0.7060998151571165
AUC Score: 0.7060998151571164
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.72      0.71       541
           1       0.71      0.69      0.70       541

   micro avg       0.71      0.71      0.71      1082
   macro avg       0.71      0.71      0.71      1082
weighted avg       0.71      0.71      0.71      1082

Accuracy: 0.7690875232774674
AUC Score: 0.6981832997795034
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.50      0.58       171
           1       0.79      0.89      0.84       366

   micro avg       0.77      0.77      0.77       537
   macro avg       0.74      0.70      0.71       537
weighted avg       0.76      0.77      0.76       537

Accuracy: 0.7388809182209469
AUC Score: 0.6753034547152195
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.47      0.5